In [53]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error
import warnings
import os
from tqdm import tqdm
import pickle
import joblib
from statsmodels.tsa.statespace.sarimax import SARIMAXResults

warnings.filterwarnings("ignore")

In [54]:
# --- 1. Load and Aggregate Data ---
csv_path = "sensor_data.csv"  
df = pd.read_csv(csv_path, parse_dates=['timestamp'])
df = df.sort_values('timestamp').set_index('timestamp')
df = df.fillna(method='ffill').fillna(0)

print(f"Original data: {len(df)} records at {pd.infer_freq(df.index)} frequency")

# Aggregate to 15-minute intervals
def aggregate_data(df, freq='15T'):
    sensor_cols = [c for c in df.columns if c not in ['hour', 'minute', 'day', 'month', 'weekday', 'is_weekend']]
    
    df_agg = df[sensor_cols].resample(freq).sum()
    
    # Recreate time features
    df_agg['hour'] = df_agg.index.hour
    df_agg['minute'] = df_agg.index.minute
    df_agg['day'] = df_agg.index.day
    df_agg['month'] = df_agg.index.month
    df_agg['weekday'] = df_agg.index.weekday
    df_agg['is_weekend'] = df_agg.index.weekday.isin([5, 6]).astype(int)
    
    return df_agg

df = aggregate_data(df, '15T')
print(f"Aggregated data: {len(df)} records at {pd.infer_freq(df.index)} frequency")

Original data: 2400 records at 3min frequency
Aggregated data: 480 records at 15min frequency


In [55]:

# --- 2. Separate sensor columns from time columns ---
time_cols = ['hour', 'minute', 'day', 'month', 'weekday', 'is_weekend']
sensor_cols = [c for c in df.columns if c not in time_cols]

print(f"✅ Processing {len(sensor_cols)} sensors at 15-minute resolution")
print(f"Sample sensors: {sensor_cols[:5]}...")

# --- 3. Detect frequency and set parameters ---
freq = pd.infer_freq(df.index)
step_minutes = 15  # We know this now
SEASONAL_PERIODS = int(24 * 60 / step_minutes)  # 96 for 15-min data
FORECAST_STEPS = min(SEASONAL_PERIODS, 96)  # Forecast one day max

print(f"⏱ 15-minute data → Seasonal period = {SEASONAL_PERIODS}")


✅ Processing 74 sensors at 15-minute resolution
Sample sensors: ['CMSA-GAKH-01_0', 'CMSA-GAKH-01_180', 'CMSA-GAWW-11_120', 'CMSA-GAWW-11_300', 'CMSA-GAWW-12_115']...
⏱ 15-minute data → Seasonal period = 96


In [56]:
# --- 4. Improved SARIMAX modeling ---
def create_future_time_features(last_timestamp, steps, freq='15T'):
    """Create time features for future predictions without data leakage"""
    future_dates = pd.date_range(
        start=last_timestamp + pd.Timedelta(freq),
        periods=steps,
        freq=freq
    )
    
    return pd.DataFrame({
        'hour': future_dates.hour,
        'minute': future_dates.minute,
        'day': future_dates.day,
        'month': future_dates.month,
        'weekday': future_dates.weekday,
        'is_weekend': future_dates.weekday.isin([5, 6]).astype(int)
    }, index=future_dates)

In [60]:
def save_updateable_model(fit, filename, compression=True):
    """Save model in updateable format with size optimization"""
    
    # Remove training data references but keep internal state
    if hasattr(fit, 'data'):
        fit.data = None
    if hasattr(fit.model, 'data'): 
        fit.model.data = None
    
    # Keep these CRITICAL for updating:
    # - fit.filter (Kalman filter state)
    # - fit.model.endog (recent endogenous observations)  
    # - fit.model.exog (recent exogenous observations)
    # - fit.params (model parameters)
    
    if compression:
        with gzip.open(filename, 'wb') as f:
            pickle.dump(fit, f, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(filename, 'wb') as f:
            pickle.dump(fit, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_and_update_model(model_path, new_observations, new_exog=None):
    """Load model and update with new data"""
    with gzip.open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # Update model with new data
    updated_model = model.extend(new_observations, exog=new_exog)
    
    return updated_model

# --- 5. Optimized forecast loop ---
results = []
successful_models = 0

# Create the trained_models directory if it doesn't exist
os.makedirs("trained_models", exist_ok=True)
os.makedirs("trained_models_metadata", exist_ok=True)


for col in tqdm(sensor_cols, desc="Forecasting sensors"):
    y = df[col]
    
    # Skip sensors with no variance
    if y.std() < 0.01:
        print(f"⏭️ Skipping {col} (low variance)")
        continue
        
    # Ensure sufficient data
    if len(y) < 2 * SEASONAL_PERIODS:
        print(f"⏭️ Skipping {col} (insufficient data: {len(y)} points)")
        continue
    
    train_size = max(int(0.8 * len(y)), len(y) - 7 * SEASONAL_PERIODS)
    train_y, test_y = y.iloc[:train_size], y.iloc[train_size:train_size + FORECAST_STEPS]
    
    if len(test_y) == 0:
        test_y = y.iloc[train_size:train_size + FORECAST_STEPS]
    
    # Handle exogenous variables properly
    train_exog = df[time_cols].iloc[:train_size]
    future_exog = create_future_time_features(train_y.index[-1], len(test_y), '15T')
    
    try:
        # Use simpler model for speed
        model = SARIMAX(
            train_y,
            exog=train_exog,
            order=(1, 0, 1), 
            seasonal_order=(0, 1, 1, SEASONAL_PERIODS), 
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        fit = model.fit(disp=False, maxiter=60)
        
        # Forecast
        forecast = fit.get_forecast(steps=len(test_y), exog=future_exog)
        forecast_values = forecast.predicted_mean
        forecast_values[forecast_values < 0] = 0
        
        mae = mean_absolute_error(test_y.values, forecast_values)
        if mae > 200:
            print(f"🔄 Retraining {col} with stationarity enforcement (MAE={mae:.2f})")

            model = SARIMAX(
                train_y,
                exog=train_exog,
                order=(1, 0, 1),           # Simpler non-seasonal part
                seasonal_order=(0, 1, 1, SEASONAL_PERIODS),  # Simplified seasonal
                enforce_stationarity=True,
                enforce_invertibility=False
            )
            fit = model.fit(disp=False, maxiter=50)
        
            forecast = fit.get_forecast(steps=len(test_y), exog=future_exog)
            forecast_values = forecast.predicted_mean
            forecast_values[forecast_values < 0] = 0
            mae = mean_absolute_error(test_y.values, forecast_values)

        # Store results
        result_df = pd.DataFrame({
            'timestamp': test_y.index,
            'actual': test_y.values,
            'forecast': forecast_values.values,
            'sensor': col,
            'mae': mae
        })
        results.append(result_df)
        successful_models += 1
        
        # Save the trained model
        model_filename = f"trained_models/{col.replace('/', '_')}_model.pkl"
        fit.save(model_filename, True)  # statsmodels native saving
        
        # Also save metadata for real-time forecasting
        metadata = {
            'last_training_timestamp': train_y.index[-1],
            'training_data_end': len(train_y),
            'sensor_name': col,
            'mae': mae,
            'model_order': (1, 0, 1),
            'seasonal_order': (0, 1, 1, 96)
        }
         # Optional: Plot first few sensors
        if successful_models <= 80:
            plt.figure(figsize=(12, 4))
            plt.plot(train_y.index[-100:], train_y[-100:], label='Train', color='steelblue')
            plt.plot(test_y.index, test_y.values, label='Test', color='black')
            plt.plot(test_y.index, forecast_values.values, label='Forecast', linestyle='--', color='red')
            plt.title(f"{col} (MAE={mae:.2f}) - 15-min Aggregated")
            plt.legend()
            plt.tight_layout()
            plt.savefig(f"indiv_plots_final/forecast_15min_{col.replace('/', '_')}.png", dpi=150)
            plt.close()

        with open(f"trained_models_metadata/{col.replace('/', '_')}_metadata.pkl", 'wb') as f:
            pickle.dump(metadata, f)
       

        
            
    except Exception as e:
        print(f"❌ {col}: {str(e)[:100]}...")

print(f"\n✅ Completed: {successful_models}/{len(sensor_cols)} sensors successfully modeled")

# --- 6. Save results ---
if results:
    all_forecasts = pd.concat(results, ignore_index=True)
    all_forecasts.to_csv("sensor_forecasts_15min.csv", index=False)
    print("✅ Forecasts saved to: sensor_forecasts_15min.csv")

Forecasting sensors:   3%|▎         | 2/74 [05:57<3:37:53, 181.58s/it]

🔄 Retraining CMSA-GAWW-11_120 with stationarity enforcement (MAE=2633.02)


Forecasting sensors:   4%|▍         | 3/74 [9:29:41<224:42:34, 11393.72s/it]


KeyboardInterrupt: 

In [59]:
import pandas as pd
import pickle
from statsmodels.tsa.statespace.sarimax import SARIMAXResults
import os

def load_and_forecast(model_path, future_steps, future_exog):
    """Load a saved model and generate forecasts"""
    try:
        # Load the model
        loaded_fit = SARIMAXResults.load(model_path)
        print(f"✅ Model loaded successfully from {model_path}")
        
        # Generate forecast
        print(type(loaded_fit))
        forecast_result = loaded_fit.get_forecast(steps=future_steps, exog=future_exog)
        forecast_values = forecast_result.predicted_mean
        confidence_intervals = forecast_result.conf_int()
        
        # Ensure non-negative forecasts
        forecast_values[forecast_values < 0] = 0
        
        return forecast_values, confidence_intervals
        
    except Exception as e:
        print(f"❌ Error loading/forecasting: {e}")
        return None, None

# Example usage:
def test_loaded_model(sensor_name, future_hours=2):
    """Test loading and forecasting for a specific sensor"""
    
    # Model path
    safe_name = sensor_name.replace('/', '_')
    model_path = f"trained_models/{safe_name}.pkl"
    
    if not os.path.exists(model_path):
        print(f"❌ Model file not found: {model_path}")
        return None
    

    
    # Create future time features (same as during training)
    future_steps = int(future_hours * 60 / 15)  # 15-min intervals
    last_timestamp = pd.Timestamp('2025-08-24 00:00')  # Or use your reference time
    
    future_exog = create_future_time_features(last_timestamp, future_steps, '15T')
    
    # Load and forecast
    forecasts, confidence = load_and_forecast(model_path, future_steps, future_exog)
    
    if forecasts is not None:
        print(f"📈 {sensor_name} - Next {future_hours} hours forecast:")
        for i, (timestamp, value) in enumerate(zip(future_exog.index, forecasts)):
            if i < 8:  # Show first 2 hours
                print(f"   {timestamp}: {value:.1f} people")
        return forecasts
    else:
        print(f"❌ Failed to forecast for {sensor_name}")
        return None
    
test_loaded_model("model1", 2)

# Your existing function (make sure it's available)
def create_future_time_features(last_timestamp, steps, freq='15T'):
    """Create time features for future predictions"""
    future_dates = pd.date_range(
        start=last_timestamp + pd.Timedelta(freq),
        periods=steps,
        freq=freq
    )
    
    return pd.DataFrame({
        'hour': future_dates.hour,
        'minute': future_dates.minute,
        'day': future_dates.day,
        'month': future_dates.month,
        'weekday': future_dates.weekday,
        'is_weekend': future_dates.weekday.isin([5, 6]).astype(int)
    }, index=future_dates)

✅ Model loaded successfully from trained_models/model1.pkl
<class 'statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper'>
❌ Error loading/forecasting: object of type 'NoneType' has no len()
❌ Failed to forecast for model1


In [ ]:
import pandas as pd
import pickle
from statsmodels.tsa.statespace.sarimax import SARIMAXResults
import os

def load_and_forecast(model_path, future_steps, future_exog):
    """Load a saved model and generate forecasts"""
    try:
        # First, check what type of object we're loading
        with open(model_path, 'rb') as f:
            loaded_obj = pickle.load(f)
        
        print(f"📦 Loaded object type: {type(loaded_obj)}")
        
        # Handle different object types
        if isinstance(loaded_obj, dict):
            print("❌ This is a metadata dictionary, not a model")
            print(f"   Keys in dictionary: {list(loaded_obj.keys())}")
            return None, None
            
        elif hasattr(loaded_obj, 'get_forecast'):
            # This is a proper SARIMAXResults object
            print(f"✅ Proper model loaded from {model_path}")
            
            # Generate forecast
            forecast_result = loaded_obj.get_forecast(steps=future_steps, exog=future_exog)
            forecast_values = forecast_result.predicted_mean
            confidence_intervals = forecast_result.conf_int()
            
            # Ensure non-negative forecasts
            forecast_values[forecast_values < 0] = 0
            
            return forecast_values, confidence_intervals
            
        else:
            print(f"❌ Unknown object type: {type(loaded_obj)}")
            return None, None
        
    except Exception as e:
        print(f"❌ Error loading/forecasting: {e}")
        return None, None

# Alternative: Direct SARIMAXResults loading
def load_model_directly(model_path):
    """Load model using statsmodels native method"""
    try:
        model = SARIMAXResults.load(model_path)
        print(f"✅ Model loaded directly: {type(model)}")
        return model
    except Exception as e:
        print(f"❌ Direct loading failed: {e}")
        return None

def forecast_with_model(model, future_steps, future_exog):
    """Generate forecast with a loaded model"""
    try:
        forecast_result = model.get_forecast(steps=future_steps, exog=future_exog)
        forecast_values = forecast_result.predicted_mean
        forecast_values[forecast_values < 0] = 0
        return forecast_values
    except Exception as e:
        print(f"❌ Forecasting failed: {e}")
        return None
    
test_loaded_model("model1", 2)